# **LaTeX table with the BAO-fit results**

This notebook shows how to create a LaTeX table with the BAO-fit results varying the settings

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from itertools import product
import os
from utils_baofit import BAOFitInitializer

# dataset_list = ["DESY6", "DESY6_dec<-23.5", "DESY6_dec>-23.5", "DESY6_DR1tiles_noDESI"]
dataset_list = ["DESY6", "DESY6_dec<-23.5", "DESY6_dec>-23.5", "DESY6_DR1tiles_noDESI", "DESY6_DR1tiles_DESIonly"]
include_wiggles_list = ["y"]
weight_type_list = [1]
bins_removed_list = [[], [0], [1], [2], [3], [4], [5], [1,2,3,4,5], [0,2,3,4,5], 
                     [0,1,3,4,5], [0,1,2,4,5], [0,1,2,3,5], [0,1,2,3,4]]

fit_results = {}

for dataset, include_wiggles, weight_type, bins_removed in product(dataset_list, include_wiggles_list, weight_type_list, bins_removed_list):
    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = include_wiggles
            self.dataset = dataset
            # self.dataset = "DESY6_dec<-23.5"
            # self.dataset = "DESY6_dec>-23.5"
            self.weight_type = weight_type
            self.mock_id = "mean" # it will not be used...
            self.nz_flag = "fid"
            # self.nz_flag = "clusteringz"
            self.cov_type = "cosmolike"
            self.cosmology_template = "planck"
            self.cosmology_covariance = "planck"
            self.delta_theta = 0.2
            self.theta_min = 0.5
            self.theta_max = 5
            self.n_broadband = 3
            self.bins_removed = bins_removed
            self.diag_only = "n"
            self.remove_crosscov = "n"
            self.alpha_min = 0.8
            self.alpha_max = 1.2
            self.base_path = None
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"
    
    # 4. BAO fit initializer. This basically creates the path to save the results
    baofit_initializer = BAOFitInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag,
        cov_type=args.cov_type,
        cosmology_template=args.cosmology_template,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        n_broadband=args.n_broadband,
        bins_removed=args.bins_removed,
        alpha_min=args.alpha_min,
        alpha_max=args.alpha_max,
        verbose=False,
        base_path=args.base_path,
    )

    fit_results[dataset, include_wiggles, weight_type, str(bins_removed)] = np.loadtxt(os.path.join(baofit_initializer.path_baofit, "fit_results.txt"))


In [ ]:
import pandas as pd

data = []
for key, values in fit_results.items():
    dataset, include_wiggles, weight_type, bins_removed = key
    alpha, sigma_alpha, chi2, dof = values
    data.append([dataset, weight_type, bins_removed, alpha, sigma_alpha, chi2, int(dof)])

df = pd.DataFrame(data, columns=["Dataset", "Weight Type", "Bins Removed", r"$\alpha$", r"$\sigma_\alpha$", r"$\chi^2$", "dof"])

df[r"$\alpha$"] = df[r"$\alpha$"].map(lambda x: f"{x:.3f}")
df[r"$\sigma_\alpha$"] = df[r"$\sigma_\alpha$"].map(lambda x: f"{x:.3f}")
df[r"$\chi^2$"] = df[r"$\chi^2$"].map(lambda x: f"{x:.1f}")
df["dof"] = df["dof"].astype(int)

latex_table = df.to_latex(index=False, escape=False, column_format="ccc|cccc")

print(latex_table)
